In [99]:
%load_ext autoreload
%autoreload 2

from excel_matches_livingscores import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# 1. load matches

In [100]:
with open('matches_excel.json') as f:
    matches = json.load(f)
print(len(matches))

91035


# 2. Load Booking data

In [102]:
print("Loading Booking hotels")
bkgh = load_booking(matches.values())
print(len(bkgh))

Loading Booking hotels
91035


# 2.b Load Amadeus spreadsheet

In [103]:
print("Loading Amadeus hotels")
amdh_full = pd.read_excel('./Amadeus All Properties - FEB 2017 17022017 .xlsx', header=0, index_col='PROPERTY_CODE', skiprows=1)
amdh = convert_amd_df_to_matching_format(amdh_full)
amdh.amd_id = amdh.amd_id.values.astype(str)
def pc_to_bkg(pc):
    return matches.get(pc)
amdh['bkg_id'] = amdh['amd_id'].apply(pc_to_bkg)

namdh = len(amdh)
print("Loaded %d hotels" % namdh)

Loading Amadeus hotels
Loaded 140709 hotels


# 3. Create matches DF and compute features

In [104]:
amdh.bkg_id = amdh.amd_id.apply(lambda i: matches[i] if i in matches else None)
matches_df = amdh.join(bkgh, on='bkg_id', how='inner', rsuffix='_bkg')
matches_df = matches_df[['amd_id', 'bkg_id_bkg',u'name', 'name_bkg', u'chain', 'chain_bkg', 'lat',  u'lng', u'lat_bkg', u'lng_bkg']]
mdf = matches_df
mdf.chain_bkg.fillna('', inplace=True)

mdf.name = mdf.name.apply(normalize)
mdf.name_bkg = mdf.name_bkg.apply(normalize)
mdf.chain = mdf.chain.astype(unicode).apply(normalize)
mdf.chain_bkg = mdf.chain_bkg.astype(unicode).apply(normalize)

In [105]:
s = mdf

In [106]:
s.chain.fillna('', inplace=True)
s.chain_bkg.fillna('', inplace=True)
s.name.fillna('', inplace=True)
s.name_bkg.fillna('', inplace=True)

In [107]:
def _get_geo_dist(lat1, lng1, lat2, lng2):
    """
        Geographic distance using Haversine formula
    """
    r = 6371000  # Earth radius in meters
    phi1 = math.radians(lat1)
    phi2 = math.radians(lat2)
    lambda1 = math.radians(lng1)
    lambda2 = math.radians(lng2)
#     haversin = lambda theta: (1 - math.cos(theta)) / 2.0
    def haversin(theta):
        return (1 - math.cos(theta)) / 2.0
    h = haversin(phi2 - phi1) + math.cos(phi1) * math.cos(phi2) * \
        haversin(lambda2 - lambda1)
    c = 2 * math.atan2(math.sqrt(h), math.sqrt(1 - h))
    
    return r * c

vect_geo_dist = np.vectorize(_get_geo_dist)

def row_name_sim(row):
    return get_name_sim(row['name'], row['name_bkg'])

def row_name_sim_sw(row):
    return get_name_sim(row['name'], row['name_bkg'], swap_words=True)

def row_chain_sim(row):
    return get_name_sim(row['chain'], row['chain_bkg'])

def row_chain_sim_sw(row):
    return get_name_sim(row['chain'], row['chain_bkg'], swap_words=True)

def row_name_included(row):
    n = normalize(row['name']).replace(' ', '')    
    nb = normalize(row['name_bkg']).replace(' ', '')
    if not n:
        return False
    if not nb:
        return False
    return (n in nb) or (nb in n)    

def row_chain_included(row):
    n = normalize(row['chain']).replace(' ', '')    
    nb = normalize(row['chain_bkg']).replace(' ', '')
    if not n:
        return False
    if not nb:
        return False
    return (n in nb) or (nb in n)

In [108]:
s['dist'] = vect_geo_dist(s.lat, s.lng, s.lat_bkg, s.lng_bkg)
s['name_sim'] = s.apply(row_name_sim, axis=1)
s['name_sim_sw'] = s.apply(row_name_sim_sw, axis=1)
s['chain_sim'] = s.apply(row_chain_sim, axis=1)
s['chain_sim_sw'] = s.apply(row_chain_sim_sw, axis=1)

In [113]:
s['name_included'] = s.apply(row_name_included, axis=1)
s['chain_included'] = s.apply(row_chain_included, axis=1)

In [114]:
s.head()

,amd_id,bkg_id_bkg,name,name_bkg,chain,chain_bkg,lat,lng,lat_bkg,lng_bkg,dist,name_sim,name_sim_sw,chain_sim,chain_sim_sw,name_included,chain_included
PROPERTY_CODE,,,,,,,,,,,,,,,,,
YXBNAMHN,503279101.0,78347,21c museum nashville,omni nashville,synxis,omni,36.1654,-86.77745,36.157675,-86.775598,873.801206,0.647059,0.782609,0.4,0.4,False,False
HSSHABTA,700118631.0,375223,24k intl people square,jitai people s square branch,hrs gds,,31.2273,121.46843,31.222894,121.462784,725.297065,0.640000,0.780488,0.0,1.0,False,False
WVICT179,700114177.0,490087,316,316,travelclick,,37.701,-97.33467,37.700745,-97.334726,26.618877,1.000000,1.000000,0.0,1.0,True,False
ILSYRSHL,503483969.0,1674989,34 state luxury ste,34 state luxury ste skaneateles,innlink res svcs,,42.9479,-76.42683,42.947930,-76.427168,27.567067,0.760000,1.000000,0.0,1.0,True,False
HSISTCPA,502664286.0,470576,35 tam,35 tam apart,hrs gds,,41.0388,28.98274,41.038592,28.982985,32.619504,0.666667,1.000000,0.0,1.0,True,False


In [115]:
s.to_pickle('matches_cands.pickle')